In [1]:
from pysap.plugins.mri.reconstruct_3D.fourier import NUFFT, NFFT3, FFT3
import numpy as np
from pysap.plugins.mri.reconstruct_3D.utils import convert_mask_to_locations_3D

/home/bs255482/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters



                 .|'''|       /.\      '||'''|,
                 ||          // \\      ||   ||
'||''|, '||  ||` `|'''|,    //...\\     ||...|'
 ||  ||  `|..||   .   ||   //     \\    ||
 ||..|'      ||   |...|' .//       \\. .||
 ||       ,  |'
.||        ''

Package version: 0.0.3

License: CeCILL-B

Authors: 

Antoine Grigis <antoine.grigis@cea.fr>
Samuel Farrens <samuel.farrens@gmail.com>
Jean-Luc Starck <jl.stark@cea.fr>
Philippe Ciuciu <philippe.ciuciu@cea.fr>

Dependencies: 

nibabel        : >=2.1.0   - required | 2.2.1     installed
matplotlib     : >=2.1.2   - required | 2.2.2     installed
modopt         : >=1.1.4   - required | 1.1.5     installed
numpy          : >=1.11.0  - required | 1.14.5    installed
scipy          : >=1.0.0   - required | 1.1.0     installed
future         : >=0.16.0  - required | 0.16.0    installed
pysparse       : >=0.0.1   - required | 0.0.1     installed
pyqtgraph      : >=0.10.0  - required | 0.10.0    installed
progressbar    : >=3.34.3  - re

In [2]:
_mask = np.random.randint(2, size=(64,64,64))
_samples = convert_mask_to_locations_3D(_mask)
_samples_shift = convert_mask_to_locations_3D(np.fft.fftshift(_mask))
image = np.load('/volatile/bsarthou/datas/NUFFT/mri_img_2D.npy')
image = image[64:128, 64:128]
images = np.tile(image, (64,1,1))
_kspace = np.random.randn(len(_samples)) + 1j * np.random.randn(len(_samples))
print(images.shape)

(64, 64, 64)


In [3]:
def project_on_grid(kspace, mask):
    kspace_reshaped = np.zeros(mask.shape, dtype=kspace.dtype)
    axis_x, axis_y, axis_z = np.where(mask == 1)
    kspace_reshaped[axis_x, axis_y, axis_z] = kspace
    return kspace_reshaped

In [4]:
_kspace_gridded = project_on_grid(_kspace, np.fft.fftshift(_mask))

In [5]:
fourier_op_dir_nufft = NUFFT(samples=_samples, platform='cpu', shape=(64,64,64), Kd=64, Jd=3)
fourier_op_dir_nfft = NFFT3(samples=_samples, shape=(64,64,64))
fourier_op_dir_fft = FFT3(samples=_samples_shift, shape=(64,64,64))

Creating the NUFFT object...


In [6]:
fourier_op_adj_nufft = NUFFT(samples=_samples, platform='cpu', shape=(64,64,64), Kd=64, Jd=3)
fourier_op_adj_nfft = NFFT3(samples=_samples, shape=(64,64,64))
fourier_op_adj_fft = FFT3(samples=_samples_shift, shape=(64,64,64))

Creating the NUFFT object...


In [7]:
kspace_nfft = fourier_op_dir_nfft.op(images)
kspace_nufft = fourier_op_dir_nufft.op(images)
kspace_fft = fourier_op_dir_fft.op(images)
# kspace_fft = kspace_fft.flatten()/np.sqrt(64*64*64)

In [8]:
print(kspace_nufft.shape)
print(kspace_nfft.shape)
print(kspace_fft.shape)

(130928,)
(130928,)
(64, 64, 64)


In [9]:
img_nfft = fourier_op_adj_nfft.adj_op(_kspace)
img_nufft = fourier_op_adj_nufft.adj_op(_kspace)
img_fft = fourier_op_adj_fft.adj_op(_kspace_gridded)

### Check the adjoint relation

In [10]:
rslt_1_nfft = np.dot(kspace_nfft, np.conj(_kspace))
rslt_2_nfft = np.dot(images.flatten(), np.conj(img_nfft.flatten()))
rslt_1_nufft = np.dot(kspace_nufft, np.conj(_kspace))
rslt_2_nufft = np.dot(images.flatten(), np.conj(img_nufft).flatten())

In [11]:
rslt_1_fft = np.dot(kspace_fft.flatten(), np.conj(_kspace_gridded.flatten()))
rslt_2_fft = np.dot(images.flatten(), np.conj(img_fft.flatten()))

In [12]:
print(rslt_1_nfft/rslt_2_nfft)
print((rslt_1_nufft/rslt_2_nufft))
print(rslt_1_fft / rslt_2_fft)

(0.999999999999951+5.1565958756268535e-15j)
(262144.0016518826+0.0014689325528945147j)
(1.0000000000000195-9.696153813185675e-15j)
